## Sampling with weights in Pandas

- sampling core utilities is based on numpy (see docstring)
- [file](https://github.com/pandas-dev/pandas/blob/49d371364b734b47c85733aac74b03ac4400c629/pandas/core/sample.py) containing sampling functions

## Some random data

In [ ]:
from vaep.utils import create_random_df
X = create_random_df(100, 15, prop_na=0.1).stack().to_frame(
    'intensity').reset_index()

freq = X.peptide.value_counts().sort_index()
freq.name = 'freq'

X = X.set_index(keys=list(X.columns[0:2]))  # to_list as an alternative
freq

In [ ]:
X

In [ ]:
print(f"Based on total number of rows, 95% is roughly: {int(len(X) * 0.95)}")
print("Based on each sample's 95% obs, it is roughly: {}".format(
    X.groupby('Sample ID').apply(lambda df: int(len(df) * 0.95)).sum()))

## Samling using a column with the weights

In [ ]:
X = X.join(freq, on='peptide')
X

In [ ]:
t = X.groupby('Sample ID').get_group('sample_003')
t

In [ ]:
t.sample(frac=0.75, weights='freq')

Sampling the entire DataFrame based on the freq will normalize on N of all rows. The normalization leaves relative frequency the same (if no floating point unprecision is reached)

In [ ]:
# number of rows not the same as when using groupby (see above)
X.sample(frac=0.95, weights='freq')

### Sampling fails with groupby, reindexing needed

The above is not mapped one to one to the groupby sample method. One needs to apply it to every single df.

In [ ]:
# X.groupby('Sample ID').sample(frac=0.95, weights='freq') # does not work
X.groupby('Sample ID').apply(
    lambda df: df.reset_index(0, drop=True).sample(frac=0.95, weights='freq')
).drop('freq', axis=1)

And passing a Series need the original X to be indexed the same (multi-indices are not supported)

In [ ]:
# for i, t in X.groupby('Sample ID'):
#     t = t.sample(frac=0.75, weights=freq)
# t

In [ ]:
X = X.reset_index('Sample ID')
X

In [ ]:
X.groupby(by='Sample ID').sample(frac=0.95, weights=freq)

In [ ]:
X.groupby(by='Sample ID').get_group('sample_002')

## Sanity check: Downsampling the first feature

In [ ]:
freq.loc['feat_00'] = 1  # none should be selected

In [ ]:
freq = freq / freq.sum()
freq

In [ ]:
X.groupby(by='Sample ID').sample(
    frac=0.5, weights=freq).sort_index().reset_index().peptide.value_counts()

## Using a series

- in the above approach, sampling weights might be readjusted based on the values present in `sample` as `NAN`s lead to the weights not summing up. Alteratively one could loop through the wide format rows and sample values from these.

In [ ]:
freq

In [ ]:
X = X.drop('freq', axis=1).set_index(
    'Sample ID', append=True).squeeze().unstack(0)
X

In [ ]:
X.iloc[0].sample(frac=0.8, weights=freq).sort_index()

Sampling using the wide format would garuantee that the weights are not adjusted based on missing values, but that instead missing values are sample into on or the other set. Ultimately `NaN`s are dropped also in this approach.

In [ ]:
import pandas as pd
data = {}
for row_key in X.index:
    data[row_key] = X.loc[row_key].sample(frac=0.8, weights=freq)
pd.DataFrame(data).stack()